## Author: Mayank Yadav

---

### Overview

This module provides functions to fetch data from various APIs across multiple domains.

### Supported API Domains

1. **Police and Crime**
2. **Agriculture**
    - Crops
    - Fertilizers
    - Irrigation
    - Planning
    - Soil & Water Conservation  
3. **Water**
4. **Education**
5. **Economy**
6. **Governance and Administration**
7. **Industry**
    - Tourism
    - MSMEs
8. **Infrastructure**
    - Roads
    - Railways
9. **Labour and Employment**
10. **Rural Development**


In [18]:

import os
import sys
import time
import re
import json
import threading
import itertools
from concurrent.futures import ThreadPoolExecutor, as_completed

import requests
import pandas as pd

from typing import Dict, Any


api_key = "579b464db66ec23bdd000001948d55cf357f47cc74b80568eae5c51a"

In [19]:
crime_data_apis = {
    "State/UT-wise Number of Indian Penal Code (IPC) Crimes from 2020 to 2022": "/resource/15150682-a9ed-475d-b0e3-67b292e90d22",
    
# Metropolitan Cities
    
    # age and gender
    "City, Age Group and Gender-wise Persons Arrested under IPC Crimes in Metropolitan Cities during 2019": "/resource/f28fb4fd-86cb-49e5-a222-6776bf371fdd",
    "Age Group and Gender-wise Persons Arrested under IPC Crimes (in Metropolitan Cities) during 2020": "/resource/28c1b00d-3d75-46e8-ab03-f1c7bbaff11c",
    "Age Group and Gender-wise Persons Arrested under IPC Crimes (in Metropolitan Cities) during 2021": "/resource/6377dda7-c8b3-4e9b-8d88-ff1a281ce2b0",
    "Age Group and Gender-wise Persons Arrested under SLL Crimes (in Metropolitan Cities) during 2021": "/resource/219730ea-c97d-4e98-a62a-17876774c85b",
    
    # against women
    "City & Crime Head-wise IPC and SLL Crimes against Women in Metropolitan Cities during 2017": "/resource/5e7e25eb-4cc6-475d-893f-40fc50884eff",
    "Crime Head-wise Disposal of Persons Arrested under Indian Penal Code (IPC) Crimes in Metropolitan Cities during 2021": "/resource/4f17823c-dd32-460a-9b33-a4b2b4402bf4",
    
    "Crime Head-wise Police Disposal of Crime against Women in Metropolitan Cities during 2017": "/resource/c8b12c25-e68f-4a27-a10f-3fb3cf05f4e3",
    "Crime Head-wise Police Disposal of Crime against Women in Metropolitan Cities during 2021": "/resource/e45d1d52-cfb0-462f-8da6-23bda260077c",
    "Crime Head-wise Police Disposal of Crime against Women in Metropolitan Cities during 2020": "/resource/b8cd95ac-9bec-46a1-a94d-2fab4a66cb2a",
    "Crime Head-wise Police Disposal of Crime against Women in Metropolitan Cities during 2019": "/resource/9ba5ccdb-25de-4e03-8985-49cb06bb1662",
    "Crime Head-wise Police Disposal of Crime against Women in Metropolitan Cities during 2018":"/resource/b7311014-31af-47da-b006-7fa4a38ca64b",
    
    "Crime Head-wise Court Disposal of Crime against Women in Metropolitan Cities during 2017": "/resource/e64f41ef-92f9-437a-9cc4-253db907aacf",
    "Crime Head-wise Court Disposal of Crime against Women in Metropolitan Cities during 2021": "/resource/a21dcf9c-c4e4-4973-bf9f-7a7dadccd7c0",
    "Crime Head-wise Court Disposal of Crime against Women in Metropolitan Cities during 2020": "/resource/ec646c30-45a4-4189-9b82-2ec27e5549fa",
    "Crime Head-wise Court Disposal of Crime against Women in Metropolitan Cities during 2019": "/resource/e25348f1-2186-4880-a133-b7488a7b1d1c",
    "Crime Head-wise Court Disposal of Crime against Women in Metropolitan Cities during 2018": "/resource/e7bc51a4-26c2-4c05-8f39-e338cdcfc66f",





    # crime head, age, and gender
    "Crime Head-wise Age Group and Gender-wise Persons Arrested under IPC Crimes in Metropolitan Cities during 2021": "/resource/6dccff77-d7ee-4dab-95d8-c4961b2eed48",
    "Crime Head-wise Age Group and Gender-wise Persons Arrested under SLL Crimes in Metropolitan Cities during 2021": "/resource/b2c4fbfe-4d22-4496-ad35-8b2238f29083",
    
    # crime head wise  IPC and SLL
    "State/City-wise Disposal of Persons Arrested under Indian Penal Code and Special & Local Laws (IPC & SLL) Crimes (in Metropolitan Cities) during 2021": "/resource/1a941c5e-f1f4-4c3e-8615-74850875f614",
    

# Court Disposals
    "Crime Head-wise Court Disposal of IPC Crime Cases during 2021": "/resource/2f29bd18-59d8-4f3e-9888-c6a0501a738e",
    "Crime Head-wise Court Disposal of SLL Crime Cases during 2021": "/resource/592e4555-9ba1-421c-aa36-9ae3a4636757",
    "State/UT-wise Court Disposal of IPC Crime Cases during 2021": "/resource/6f13237f-f727-4af9-97f1-6a417209b19e",
    "State/UT-wise Court Disposal of SLL Crime Cases during 2021": "/resource/b1f559c0-6a1b-4214-88b5-aaff2188f088",

    "Crime Head-wise Court Disposal of Crime against Women during 2017": "/resource/7c851b31-a0b3-4e49-9e88-c8bdd9d17449",
    "Crime Head-wise Court Disposal of Crime against Women during - 2021": "/resource/7f49af73-bbeb-484d-a1cc-f91981deff19",
    "Crime Head-wise Court Disposal of Crime against Women during 2020": "/resource/ddbc0642-6caf-4b3e-b141-98e1274bd14c",
    "Crime Head-wise Court Disposal of Crime against Women during 2019": "/resource/dc1fe545-076b-4d7d-9aec-0b26337ee54b",
    "Crime Head-wise Court Disposal of Crime against Women during 2018": "/resource/3c9d19e4-da1a-4df4-8062-f5f67e7881e3",



# Against Foreigners 
    "Crime Head-wise Crime against Foreigners during 2016": "/resource/c40f19dc-2f38-4bed-b093-2506622c984a",
    "Crime Head-wise Crime against Foreigners during 2018": "/resource/9056fe93-ca71-4100-b0a9-02e244313607",
    "Crime Head-wise Crime against Foreigners during 2019": "/resource/b6a43056-ea8e-48dc-b5f8-10ca22ed3c9b",
    "Crime Head-wise Crime against Foreigners during 2020": "/resource/3b30418e-f6d9-4c2a-a1d0-57762820eb41",
    "Crime Head-wise Crime against Foreigners during 2021": "/resource/f174cf36-1a1a-4b87-8c22-33741791326b",
    
# Police Disposals
    "Crime Head-wise Police Disposal of IPC Crime Cases during 2016": "/resource/4759cf71-d6dd-4b37-80b1-e0a10e45d90b",
    "Crime Head-wise Police Disposal of IPC Crime Cases during 2018": "/resource/08e68544-76b1-4596-a09e-47047abb26e2",
    "Crime Head-wise Police Disposal of IPC Crime Cases (Crime Head-wise) during 2019": "/resource/9fed9e21-d687-4c24-aa6e-63cb24bb68d3",
    "Crime Head-wise Police Disposal of IPC Crime Cases during 2020": "/resource/c73b59c9-04ed-4f03-98ed-4001d006962f",
    "Crime Head-wise Police Disposal of IPC Crime Cases during 2021": "/resource/8562bdac-58f1-40f3-9f32-b5f657e872a5",
    "Crime Head-wise Police Disposal of SLL Crime Cases during 2021": "/resource/7af701d0-53e5-42f9-aa52-8419bd30d59f",
    "Police Disposal of SLL Crimes in Metropolitan Cities during 2021": "/resource/fabf043a-e3e0-49a4-8c1e-372eba6e0370",
    "State/UT-wise Police Disposal of SLL Crime Cases during 2021": "/resource/305e23b5-67f2-4c31-8fd8-b05b49bb453b",

    "Crime Head-wise Police Disposal of Crime against Children during 2017": "/resource/e6deda82-8920-4954-841e-c4042729e971",
    "Crime Head-wise Police Disposal of Crime against Children during 2018": "/resource/bca3bd56-cd96-4df1-aec1-71ac46a9aabf",
    "Crime Head-wise Police Disposal of Crime against Children during 2019": "/resource/e0502065-0ba5-4ceb-9089-8993f9702fda",
    "Crime Head-wise Police Disposal of Crime against Children during 2020":"/resource/a2732898-e9cf-46fa-81be-8b713eb9dbfb",
    "Crime Head-wise Police Disposal of Crime against Children during 2021": "/resource/a5207328-d1ce-446c-9bbd-e65e9715a609",

    "Crime Head-wise Police Disposal of Crime against Women during 2021": "/resource/f7fd4d97-6bb0-40cd-90fb-0ed55a93c211",
    "Crime Head-wise Police Disposal of Crime against Women during 2020": "/resource/a502be00-c723-4f33-9cdf-af98d00fda21",
    "Crime Head-wise Police Disposal of Crime against Women during 2019": "/resource/c8968c89-0549-429f-a456-5ae01e51fa18",
    "Crime Head-wise Police Disposal of Crime against Women during 2018": "/resource/b7e71ce1-a8d7-48bd-8cda-c298d2a8edc3",
    "Crime Head-wise Police Disposal of Crime against Women during 2017": "/resource/8aab0d11-03ea-43bb-91f4-794d23f176d5",


# Against women
    "State/City-wise Indian Penal Code (IPC) Crimes against Women (Crime Head & City) during 2021": "/resource/eb37b5f8-671f-4a52-b528-c20d39797f43",
    "Crime against Women during 2014": "/resource/de6ddfff-c5b9-4058-a54a-776cee56567f",
    "Crime against Women during 2001-2012": "/resource/0171c62a-5304-43fa-83ff-8c9f16033e41",
    "Crime against Women during 2013": "/resource/ed286df7-e6bf-4938-8892-188cdeb1e62e",


# Against Children
    "Crime head-wise persons arrested under crime against children during 2013": "/resource/11626229-2afb-4950-8000-69d309c4269c",

# Arrests related
    "Persons Arrested under Various Sections of Cr.PC during 2021": "/resource/999d1451-4fae-48f9-9aec-586d87e9b3d6",  
    "Crime Head-wise Preventive Detentions during 2021": "/resource/75f413ea-9ccc-42ea-a2ff-f9ea7541f2b6",
    "State/UT-wise Caste of Convicts as on 31st December, 2020": "/resource/9cc08576-e504-44e8-9a63-251df05ce993",
    "State/UT-wise Religion of Convicts as on 31st December, 2020": "/resource/9241a606-688b-42b2-baea-c32916ad65cc",
    "State/UT-wise Domicile of Convicts as on 31st December, 2020": "/resource/1c64a0a2-4d9c-4bb9-8824-3b88dd9a3782",
    "State/UT-wise Education Profile of Convicts as on 31st December, 2020": "/resource/d0e86fbd-0e3f-44e8-9ac6-c412d5b66221",
    "State/UT-wise and Sex-wise Types of Prison Inmates in Special Jails as on 31st December, 2020": "/resource/7b5d0fc3-2b6c-4ade-8de8-268328f19546",
    "State/UT-wise Caste of Detenues as on 31st December, 2020": "/resource/2f742f78-6c6b-4b33-a6dd-7dfbe2efc795",
    "State/UT-wise Education Profile of other Prisoners as on 31st December, 2020": "/resource/c5d15711-07c2-4001-bb24-14b8b0ac645f",


    
# Kills
    "State/UT-wise All Uniform Police Personnel Injured on Duty during 2021": "/resource/52b4a360-991e-4683-992a-bc39b5279d21",
    "State/UT-wise All Uniform Police Personnel Killed on Duty during 2021": "/resource/c7002ab6-d7a6-4f5c-a26d-a7dbdacf92d4",
    "State/UT-wise Civilians Killed during Police Operations during 2021": "/resource/e1ce0691-9c6c-4e3e-bc62-586afb06653b",
    "All Uniform Police Personnel Killed/Injured on Duty during 2021": "/resource/7a816151-f9ce-4c19-9155-91fc6a52e6e3",

# Seizures and Recoveries
    "State/UT-wise Seizures under Arms Act during 2021": "/resource/34bf38a5-f348-4c06-a022-9408ae50eb3c",
    "State/UT-wise Seizure of Fake Indian Currency during 2021": "/resource/79c0d20c-272c-4fbd-a344-24d8304c6442",
    "Nature of Property-wise Value of Property Stolen & Recovered (in Crores) and Percentage Recovery during 2021": "/resource/56d8f4e1-098d-46ed-b1e8-4e9b054d08c6",
    "Drug - Head-wise Seizures under Narcotic Drugs and Psychotropic Substances NDPS Act during 2021": "/resource/7c4969d4-1601-4b77-8014-2d5cc0195aa5",
    "Type of Explosive-wise Seizures/Recoveries under Explosives & Explosive Substances Act during 2021": "/resource/d609a403-ee76-4f63-9abb-4ea10e99c7f3",

# Against Police
    "State/UT-wise Cases Registered against State Police Personnel during 2021": "/resource/edd8a94f-19c5-46b1-9985-2fdff15fd5aa",
    "Cases Registered against State Police Personnel for Human Right Violation during 2021": "/resource/f3ea8c03-90fd-4460-b116-447831ea2ad3",
}



In [20]:
agriculture_data_apis = {
    # "4_4.19_w_Total agriculture land holders_Others" : "/resource/15752518-baff-4672-8186-a41829c70496",
    "State/UT-wise Suicide by Persons Self-employed in Farming/agriculture during 2015 (From : Ministry of Agriculture and Farmers Welfare)" : "/resource/d2d1330d-e757-4fff-933b-80df05d41562",
    # "Production of Different Crops during 2019-20" :  "/resource/f20d7d45-e3d8-4603-bc79-15a3d0db1f9a",
    # "Area under Different Crops (Provisional) during 2019-20" : "/resource/1cb5d0c1-171a-45ec-ab6c-c02adf39982d",
    # "Rabi Summer Enrollment Claims 2016-17" : "/resource/f6702864-51ce-4981-a89f-1d57b8977cb0",
    "Area, Production and Productivity of Various Kinds of Spices from 2008-09 to 2014-15" : "/resource/62bdce72-56c6-4d12-b875-27aff49275e3",
    # "List of certified varieties of horticultural crops" : "/resource/46f587a9-7476-443e-915f-fc756a6b4e2c",
    "All India level Index Numbers of Production of Principal Crops from 2007-08 to 2015-16" : "/resource/e540df91-65d2-45a1-8b2d-b4f11023a042",
    "All India level Index Numbers of Area of Principal Crops from 2007-08 to 2015-16" : "/resource/af27a8e3-3d4f-4d67-9f2e-81730293df47",
    "Summary of Area And Production Estimates of Horticulture Crops" : "/resource/12e7661e-1b97-4f8c-a901-33844fa5412d",
    # "Average Annual Growth Rates of Major Crops" : "/resource/204f3de1-9b3f-4516-9aab-c756f5610428",
    "State-wise Area and Production of Various Horticulture Crops from 2009-10 to 2014-15" : "/resource/a2b43dcc-9cd2-4601-b183-3e859624dea4",
    "All India level Area and Production of Various Horticulture Crops from 2001-02 to 2014-15": "/resource/bbf51815-976c-449b-99c8-541641db68c8",
    # "Product wise and Month wise production of chemical Fertilizers" : "/resource/373358c8-63fd-4612-8f2b-9ce483422312",
    # "Pesticides Dealers License Report": "/resource/8db62b11-d6a1-41bd-8d9d-5d560b47b8a6",
    # "District-wise market arrivals of Paddy crop from 1978 to 2021 (April to March)": "/resource/1ec5d89e-6cff-4358-958c-67432e7a73f9",
    # "Demand and Supply of Commodities during Twelfth Plan": "/resource/27ac86aa-0352-4c13-8711-23d4720d82ea",
}

In [21]:
water_data_apis = {

    "Water Quality Affected Habitations As On 1st April 2009": "/resource/bf41d6a6-f377-4ed2-a467-020a1f2c3612",
    "Water Quality Affected Habitations As On 1st April 2011" : "/resource/d89395e0-7967-4496-9fa3-e2bb81e968cd",
    "Water Quality Affected Habitations As On 1st April 2010" : "/resource/f773b9e6-2999-4647-b4fa-752ab076ba76",
    "Water Quality Affected Habitations As On 1st April 2012" : "/resource/bd915eb6-7065-4849-a614-64377d4bc0c3",


# GJ
    "Gram Panchayat-wise Rural Sanitation Coverage in Navsari District of Gujarat as on date": "/resource/c10cfb00-c073-4141-a757-b9dbde6a8c80",
    "Gram Panchayat-wise Rural Sanitation Coverage in Narmada District of Gujarat as on date": "/resource/f9030fc2-39fe-458a-bf76-3d7e2d09f738",
    "Gram Panchayat-wise Rural Sanitation Coverage in Morbi District of Gujarat as on date": "/resource/67784b8a-b4ff-4a65-87ca-d7cbf425e714",
    "Gram Panchayat-wise Rural Sanitation Coverage in Mehsana District of Gujarat as on date": "/resource/b8f290ae-453b-4203-8525-b72a4b3163fd",
    "Gram Panchayat-wise Rural Sanitation Coverage in Mahisagar District of Gujarat as on date": "/resource/3566d6df-7b51-4347-b341-b178c81cd3c0",
    "Gram Panchayat-wise Rural Sanitation Coverage in Kheda District of Gujarat as on date": "/resource/b0246b08-1a5f-46f4-b1f7-92b6a417473c",
# GA
    "Gram Panchayat-wise Rural Sanitation Coverage in South Goa District of Goa as on date": "/resource/5fe0bfe6-0626-438e-b8b4-aa0e219ae359",
    "Gram Panchayat-wise Rural Sanitation Coverage in North Goa District of Goa as on date" : "/resource/07cc1acc-5ba9-43f4-bc71-f595d3ef58c7",
# DND
    "Gram Panchayat-wise Rural Sanitation Coverage in Dadra And Nagar Haveli District of Dadra and Nagar Haveli as on date": "/resource/11983bd3-d25f-46bc-8b5e-ee215fd6adf4",
# CG
    "Gram Panchayat-wise Rural Sanitation Coverage in Raipur District of Chhattisgarh as on date": "/resource/4d0758a2-ada0-4b2a-a6d7-7ac41587f8ef",
    "Gram Panchayat-wise Rural Sanitation Coverage in Raigarh District of Chhattisgarh as on date": "/resource/3212fcc5-faa6-42b5-a4ea-4d8fc365a9f0",
    "Gram Panchayat-wise Rural Sanitation Coverage in Mungeli District of Chhattisgarh as on date": "/resource/2c1a261f-d705-4b9e-a2b8-9137fbdde825",
    "Gram Panchayat-wise Rural Sanitation Coverage in Mahasamund District of Chhattisgarh as on date": "/resource/bcb328c7-e888-4a7c-92c0-fbc99ff32394",
    "Gram Panchayat-wise Rural Sanitation Coverage in Baloda Bazar District of Chhattisgarh as on date": "/resource/2b6d388c-421e-47b0-b87c-69665668f690",
# BR
    "Gram Panchayat-wise Rural Sanitation Coverage in Vaishali District of Bihar as on date": "/resource/ed0e9b19-982d-4465-a502-468f92ced65a",
    "Gram Panchayat-wise Rural Sanitation Coverage in Supaul District of Bihar as on date": "/resource/f1939962-ec47-4f34-829c-9f1a286a154a",
    "Gram Panchayat-wise Rural Sanitation Coverage in Sitamarhi District of Bihar as on date": "/resource/32b0b1a5-aafb-41ff-b5e7-ece13d8d1785",
    "Gram Panchayat-wise Rural Sanitation Coverage in Saharsa District of Bihar as on date": "/resource/46696f49-5ad8-4626-8811-ed91ce5352a1",
    "Gram Panchayat-wise Rural Sanitation Coverage in Patna District of Bihar as on date": "/resource/b4f1f69f-2045-437e-9fc7-077c7bf85da2",
    "Gram Panchayat-wise Rural Sanitation Coverage in Nawada District of Bihar as on date": "/resource/47f2509f-189e-4560-ad27-70325cff4685",
    "Gram Panchayat-wise Rural Sanitation Coverage in Nalanda District of Bihar as on date": "/resource/88e094c9-4315-48eb-a0a0-e518080e87cf",
    "Gram Panchayat-wise Rural Sanitation Coverage in Madhubani District of Bihar as on date": "/resource/fa6cef75-280e-4477-b7c0-e6c90552a89c",
    "Gram Panchayat-wise Rural Sanitation Coverage in Darbhanga District of Bihar as on date": "/resource/d3790b66-53c0-4583-bdf9-4425782fea53",
    "Gram Panchayat-wise Rural Sanitation Coverage in Bhojpur District of Bihar as on date": "/resource/5997dbf4-fc73-4281-a040-427c43df9f5d",
    "Gram Panchayat-wise Rural Sanitation Coverage in Bhagalpur District of Bihar as on date": "/resource/257a1786-3d29-4387-90f8-71aad73faf31",
    "Gram Panchayat-wise Rural Sanitation Coverage in Begusarai District of Bihar as on date": "/resource/7f7409d4-438b-44e7-b209-68afe993be0d",

# ASSAM
    "Gram Panchayat-wise Rural Sanitation Coverage in North Cachar Hills District of Assam as on date": "/resource/5061732f-f5f1-4262-b4e7-3814aa774fb9",
    "Gram Panchayat-wise Rural Sanitation Coverage in Sibsagar District of Assam as on date": "/resource/b77144e1-e8cd-4f9a-8c25-6af49f549782",
    "Gram Panchayat-wise Rural Sanitation Coverage in Nalbari District of Assam as on date": "/resource/e4f9d022-2244-4525-b2c7-fdb4e2b11b48",
    "Gram Panchayat-wise Rural Sanitation Coverage in Nagaon District of Assam as on date": "/resource/02641dbc-3c6f-42cd-ad1d-81d5423c7d22",
    "Gram Panchayat-wise Rural Sanitation Coverage in Marigaon District of Assam as on date": "/resource/2f1c7513-81e9-41ee-a66a-52bb4bc1ad96",
    "Gram Panchayat-wise Rural Sanitation Coverage in Lakhimpur District of Assam as on date": "/resource/ad057e7e-6595-46b8-8404-4dee0bcdcad6",
    "Gram Panchayat-wise Rural Sanitation Coverage in Kamrup Metro District of Assam as on date": "/resource/1a0ad613-2d53-4d12-8797-848155538cb6",
# arunachal
    "Gram Panchayat-wise Rural Sanitation Coverage in Lower Dibang Valley District of Arunachal Pradesh as on date": "/resource/4fcae84f-5b07-4148-9c6b-814cbdf11f4b",
    "Gram Panchayat-wise Rural Sanitation Coverage in Lohit District of Arunachal Pradesh as on date": "/resource/ef5d253b-cd66-4363-9897-b2f461d13be7",
    "Gram Panchayat-wise Rural Sanitation Coverage in East Siang District of Arunachal Pradesh as on date": "/resource/0b14fc71-ee88-4043-bbb0-73c626e0f152",
    "Gram Panchayat-wise Rural Sanitation Coverage in East Kameng District of Arunachal Pradesh as on date": "/resource/8598edd8-faaa-4e77-8bc3-1b8bcc0f5af3",
    "Gram Panchayat-wise Rural Sanitation Coverage in Dibang Valley District of Arunachal Pradesh as on date": "/resource/f44e7c98-d11a-4748-aed1-8c624bf6158e",
    "Gram Panchayat-wise Rural Sanitation Coverage in Anjaw District of Arunachal Pradesh as on date": "/resource/878e59fc-819a-42fe-b148-0ee21d1704e5",
    # AP
    "Gram Panchayat-wise Rural Sanitation Coverage in West Godavari District of Andhra Pradesh as on date": "/resource/c9ba5e59-2411-43a2-aeb5-7d0cb2bacff4",
    "Gram Panchayat-wise Rural Sanitation Coverage in Vizianagaram District of Andhra Pradesh as on date": "/resource/81aef31a-9437-4bc0-8712-f1799cff0e51",
    "Gram Panchayat-wise Rural Sanitation Coverage in Visakhapatnam District of Andhra Pradesh as on date": "/resource/a04f1c76-87bb-44b6-9d07-519e1107f195",
    "Gram Panchayat-wise Rural Sanitation Coverage in Cuddapah District of Andhra Pradesh as on date": "/resource/5093ba9e-a77f-4bda-88c3-d64e6dc2005d",
    "Gram Panchayat-wise Rural Sanitation Coverage in Anantapur District of Andhra Pradesh as on date": "/resource/3e838d1b-1c59-4fbf-bbba-b3360c26802d",
    "Gram Panchayat-wise Rural Sanitation Coverage in Faizabad District of Uttar Pradesh as on date": "/resource/e414bf4f-8d6f-4d80-9b27-756804125197",


    "Financial progress under Jal Jeevan Mission (JJM) - block Amaravathi for district Guntur, Andhra Pradesh as on date": "/resource/60350902-1902-4b76-a87c-7f558b6a2057",
    "Households with Piped Water Supply (PWS) under Jal Jeevan Mission (JJM) - block Murarai-i for district Birbhum, West Bengal as on date": "/resource/4efdf0d5-e754-492d-a9ea-0d8ab971f641",
    "Households with Piped Water Supply (PWS) under Jal Jeevan Mission (JJM) - block Billawar for district Kathua, Jammu & Kashmir as on date": "/resource/42bc9ef6-260d-4b6c-9ca6-eb7b50944837",
    "Households with Piped Water Supply (PWS) under Jal Jeevan Mission (JJM) - block Dachan for district Kishtwar, Jammu & Kashmir as on date": "/resource/31e3ad8c-07cd-4f3a-82c2-74a2ead6b932",
    "Households with Piped Water Supply (PWS) under Jal Jeevan Mission (JJM) - block Dodballapur for district Bangalore Rural, Karnataka as on date": "/resource/07627658-4372-4026-a30c-3825d12a534e",
    "Households with Piped Water Supply (PWS) under Jal Jeevan Mission (JJM) - block Chikballapur for district Chik Ballapur, Karnataka as on date": "/resource/43886708-a61a-4327-be83-5adab477d1c6",
    "Households with Piped Water Supply (PWS) under Jal Jeevan Mission (JJM) - block Panjgrain for district Rajauri, Jammu & Kashmir as on date": "/resource/8e47016b-948e-43a4-a9c6-99c0c183e5c4",
    "Households with Piped Water Supply (PWS) under Jal Jeevan Mission (JJM) - block Gund for district Ganderbal, Jammu & Kashmir as on date": "/resource/687e2808-36ac-41e4-8d70-2999fac2a349",
}

In [22]:
education_data_apis = {
# himachal
    "District wise Number of Schools and Colleges during 2020-21": "/resource/6e9fff80-5242-4672-94ee-b488bda00eb8",
    "District-wise Enrolment of Students in school during 2020-21": "/resource/897468c1-2ef1-4dbf-864f-d7f0eb4939cc",
    "Number of Teachers in Various Schools during 2020-21": "/resource/0c8c14dd-94ee-4632-b60d-374d56514c09",

# MP
    "Number of Schools by Availability of Infrastructure and Facilities, School Management and School Category in Sehore District of Madhya Pradesh (UDISE plus) during 2012-13": "/resource/68f406b8-16a7-4fe6-b568-27548e94f7e3",


# TN
    "Details of Beneficiaries under Saakshar Bharat Programme (karkkum Bharatham Thittam) by Districts in Tamil Nadu from 2010-11 to 2017-18": "/resource/dfbc330a-213b-4d1e-8a33-423d85f65a00",
    "List of the Stock of Books available at all Libraries District-wise in Tamil Nadu": "/resource/69afd0af-2cfb-442f-a1b9-806b1c8c10b9",
    "Number of Schools by Availability of Infrastructure and Facilities, School Management and School Category in Erode District of Tamil Nadu (UDISE plus) during 2012-13": "/resource/a96ad0f3-4ead-48db-9e10-2b769de2a96b",


# punjab
    "District-wise number of Girls in Classes XI to XII of Punjab from 1989 to 2022 (as on 30th September)": "/resource/bf044965-c6f3-44c6-912c-8343a4e4deee",

# assam
    "Number of Schools by Availability of Infrastructure and Facilities, School Management and School Category in Kamrup-Metro District of Assam (UDISE plus) during 2016-17": "/resource/886ba616-ba4a-499d-af79-9b9b63a26558",

# AP
    "Number of Schools by Availability of Infrastructure and Facilities, School Management and School Category in Kamle District of Arunachal Pradesh (UDISE plus) during 2018-19": "/resource/68c11f66-19d1-4804-8851-571a5c459e68",
    "Enrolment by Location, School Management, School Category and Social Category in Dibang Valley District of Arunachal Pradesh (UDISE plus) during 2016-17": "/resource/59daa40b-09de-4402-b241-3b6e65b55396",

# jharkhand
    "Enrolment by Location, School Management, School Category and Social Category in Lohardaga District of Jharkhand (UDISE plus) during 2016-17": "/resource/f024376b-f244-4e21-b87d-1a9e9637d1b7",

# J&K
    "Enrolment by Location, School Management, School Category and Social Category in Bandipora District of Jammu and Kashmir (UDISE plus) during 2016-17": "/resource/3b58879c-6e50-4198-979b-1856583034a8",

# meghalya
    "Enrolment by Location, School Management, School Category and Social Category in East Garo Hills District of Meghalaya (UDISE plus) during 2016-17": "/resource/32eb52fc-484d-4748-9c41-c07b801649fd",
    "Enrolment by Location, School Management, School Category and Social Category in East Khasi Hills District of Meghalaya (UDISE plus) during 2016-17": "/resource/51e82263-ed23-41f2-886e-e9f2a11754f1",

# haryana
    "Enrolment by Location, School Management, School Category and Social Category in Jind District of Haryana (UDISE plus) during 2015-16": "/resource/d60aa488-6bb0-4ccc-aea7-5643f456c2f1",

# WB
    "Enrolment by Location, School Management, School Category and Social Category in Dakshin Dinajpur District of West Bengal (UDISE plus) during 2015-16": "/resource/58c7abf1-1e1d-4392-9409-779bb4c69ba5",


# delhi
    "District wise Number of Schools,Student Enrollment and Pass Percentage for Class X and XII in Delhi during 2023-2024": "/resource/97adce0c-89a8-4f1e-a0f8-b66655136565",


"State/UT-wise District Score during 2021-22": "/resource/82262974-8c41-4deb-8678-f7890095face",
"Basic Information of Colleges, 2015-16": "/resource/a1f78d64-cb8d-4da9-aece-88f5fd7baf1a",
"District-wise Stage wise Teacher-Pupil Ratio of Class 11 to 12 from 1992 to 2022 (As on 30th September)":"/resource/ea1c74e0-f9a4-4e3f-abaf-a9372ef79d5f",
"District-wise Stage wise Teacher-Pupil Ratio of Class 9 to 10 from 1989 to 2022 (As on 30th September)": "/resource/a20aa01f-5ac9-4167-8a40-192ce0b1bd24",
"District-wise Stage wise Teacher-Pupil Ratio of Class 1 to 5 from 1989 to 2022 (As on 30th September)": "/resource/38e790c6-26e0-4b36-8710-ca5647c00742",


"State and UGC faculty group wise student enrollment in universities and colleges - 2012-2013": "/resource/76e6393d-611d-4987-9adb-02b967364869",
"Gross enrolment ratio - secondary School from 2006 to 2010": "/resource/231fabc7-ec68-4210-8d57-1579d056dc9c",
"GTTC Diploma Courses Data": "/resource/5e972c52-81b1-429b-9868-219aba04d0dd",
"State/UT-wise Number of Enrolments (Boys) by Management during 2021-22": "/resource/d0fb5a1f-cf7c-4e2e-8049-ea4d6a245b14",
"Educational Institutions, Scholars And Expenditure (All India)": "/resource/70331cda-6b76-4fa9-9b26-7402b8cb07a5",
"State/UT-wise Number of Schools by Management and availability of Girl Toilet and Functional Girl Toilet during 2020-21": "/resource/162415f1-9437-4d9b-b4a5-fdc2460264d1",
"Institution-wise list of programmes and discipline - 2012-2013": "/resource/eb8846f6-51f0-4bcb-ac18-ddadc70a41eb",
"Enrolment by Age and Class (UDISE plus) during 2012-13": "/resource/c5ac3c08-46a1-4940-83ef-a18d09799c48",
"State/UT-wise Number of Schools by Level of School Education during 2021-22": "/resource/dd3a1ba6-27bd-4d99-b468-832de2c582e8",
"State/UT-wise Number of Enrolment of Students - Government Schools Management and Level of School Education during 2021-22": "/resource/ff6dca55-6a53-49f1-957f-3eda5e68760b",
"Number Of Recognised Educational Institutions Of Higher Learning, All India And State wise": "/resource/3ccbd5b2-b7df-43fe-94e0-892821221f4d",
"Current enrolment across different education level in Higher Education": "/resource/e448dca6-613b-4413-af1e-337a61dcec4d",
"State/UT-wise Number of Schools by Management and Availability of Internet Facility during 2021-22": "/resource/ee728ccd-afae-4a55-81c9-3ced8ae45094",
"State/UT-wise Number of Schools by Management and Availability of Computer Facility during 2021-22": "/resource/d0ea46e1-733c-41ac-839b-2328b94cb43a",
"State/UT-wise Dropout Rate by level of Education and Gender during 2021-22": "/resource/9a8f0c4a-2c6d-4029-b005-01b163eb5a70",
"State/UT-wise Number of Enrolment of Students by Gender, All Age-group and Level of Education during 2021-22": "/resource/939f26d4-663c-4b2c-aa0b-67e87262b49d",
"State/UT-wise Percentage of Schools having Functional Toilets by Management during 2021-22": "/resource/40c05e9d-8191-4125-ae77-421ee125866a",
"Literacy Rate In India (NSSO And RGI)": "/resource/1471b575-da71-48f5-b635-f376195e3e86",
"Literacy Rate In India (State wise) upto 2011": "/resource/9fc57d32-fc04-44da-a807-6d6e1da9d9f7",
"Dropout Rates of Scheduled Tribes in 2004-05 (Provisional)": "/resource/dc89af3e-f268-4cca-b711-b252cde91356",
"School drop out rate for different age group from year 2001 to 2010": "/resource/3eed4dfb-ec70-4b7c-b23c-b55ed5dcc4cb",
"State-wise Percentage of Schools having Computer - DISE 2012-13: Flash Statistics": "/resource/e9ed0d92-ec86-4d28-96be-78dbc85ef59b",
"State-wise Percentage of Schools having Computer Aided Learning (CAL) Facility - DISE 2012-13: Flash Statistics": "/resource/8dde8fb6-ba5e-49c1-9618-4e6e61bcc69f",

"CBSE Result Statistics Class XII - 2022": "/resource/ebfb405a-686e-44d2-9f21-f3faef7246dc",
"CBSE Result Statistics Class XII - 2023": "/resource/9ebb46ba-6d80-4280-9571-5c71fabe76ad",


}

In [23]:
economy_data_apis = {
    "Wholesale Price Index (Base Year 2011-12) Upto May 2017": "/resource/abfd2d50-0d73-4a3e-9027-10edb3d21940",
    "State-wise number of establishments for all broad activity by sector and type of establishment on yearly basis": "/resource/370796b4-d67d-48a2-acb6-f3a29a3fdef8",
    "State-wise Labour Force Participation Rate (per 1000) during 2011-12": "/resource/4ae13c0a-f318-4346-82a4-3286b6e859e7",
    "Broad activity wise number and percentage distribution of persons employed in proprietary establishments by social group of the owner on yearly basis": "/resource/c97bb00f-332c-4333-a396-2f1dfc19b670",
    "State Level Consumer Price Index (Rural/Urban) Up to July 2014": "/resource/25df9c35-3409-48e9-bf76-a538a1c1d72a",
    "All India Consumer Price Index (Rural/Urban) Up to November 2016": "/resource/19ba5183-8e0e-44ab-a726-e6b89082cd60",
    "All India Consumer Price Index (Rural /Urban) up to December 2014": "/resource/16396955-74d2-43eb-80e7-8ea29fe72148",
    "All India Consumer Price Index (Rural/Urban) Upto November 2013": "/resource/2b0713d9-24fc-4231-b048-e03e287f1095",
    "State Level Consumer Price Index (Rural/Urban) Upto April 2013": "/resource/4618bd74-2467-40ae-969b-b09e0735991e",
    "All India level Cash and Capital Reconciliation Account of Government Administration and Departmental Commercial Undertakings from 2000-01 to 2015-16": "/resource/e6314a94-cb5d-4985-86b4-0f51709511a0",
    "State Level Consumer Price Index (Rural/Urban) Up to December 2015": "/resource/29abd6cc-4ab2-4ef3-8695-66693ba73b19",
    "State Level Consumer Price Index (Rural/Urban) up to February 2015": "/resource/618dc021-b211-4e43-9ef9-e6702a779eda",
    "All India Consumer Price Index (Rural /Urban) up to March 2015": "/resource/ce7292d4-cfb5-4400-be2b-0bd9e55cc287",
    "All India Consumer Price Index (Rural /Urban) up to August 2015": "/resource/4b5cb9e7-8051-4980-b68d-bfd6845cf966",
    "State Level Consumer Price Index (Rural/Urban) Upto September 2014": "/resource/55e93fc3-4e73-4ba1-a3ad-e927066db6fc",
    "Notes And Coins Issued upto 2012-13": "/resource/54cb74cb-cebf-4011-a22a-c96a84c2700b",
    "National Income at factor cost at current prices from 2004-05 to 2013-14": "/resource/503fecf6-a064-490b-a70f-5ed51bbe32a4",
    "State Level Consumer Price Index (Rural/Urban) up to April 2015": "/resource/c2ed9ff8-f73c-411a-90f8-7c5b939bf0b2",
    "State Level Consumer Price Index (Rural/Urban) Up to August 2016": "/resource/04af682a-3be2-4f6b-93e8-16e5b029c7a1",
    "State Level Consumer Price Index (Rural/Urban) Up to August 2014": "/resource/6e1f7334-151f-4d65-b9f1-6f8836ab501e",
    "All India Consumer Price Index (Rural /Urban) up to September 2015": "/resource/2ab01798-88b3-476b-815c-f02940996c61",
    "All India Consumer Price Index (Rural /Urban) up to February 2015": "/resource/ba46d48f-8763-4c05-9134-4f389a26723c",
    "All India Consumer Price Index (Rural /Urban) up to June 2015": "/resource/61464984-aebb-4aa6-bc80-537cecb480f4",
    "Quarterly Estimates of GDP at constant prices for the base year 2004-05": "/resource/0e7e1ff7-819b-4fc0-8daa-eb21a7c60f96",
    "State-wise Gross Domestic Product (GDP) at constant price on yearly basis": "/resource/b5f3f1ac-c096-493d-9f17-ac4a9843c43e",
    "Investment and Consumption Expenditure as proportion of GDP at constant 2004-05 prices": "/resource/e69a80f7-0bb5-484c-b439-b3ccd11db71a",
    "Monthly data of employment exchanges from July 2012 to April 2013": "/resource/05c95863-884d-4981-a000-8f7d603eb586",
    "All India Consumer Price Index (Rural/Urban) upto January 2019": "/resource/481b5b44-f538-4540-82c9-39c9a2f16f3a",
    "All India Consumer Price Index (Rural/Urban) upto February 2019": "/resource/6a04136f-1e9e-435f-bf17-f4e2d76ba044",
    "Annual Estimates of Rates of GDP at Market Price at Current Prices, 2004-05 Series - From 2004-05 to 2013-14 (as on 30.05.2014)": "/resource/dc7d61e2-6702-4aaf-82c0-3540d09a2dde",
    "Gross State Domestic Product at Current Prices - From 2004-05 to 2013-14 (as on 01.03.2014)": "/resource/af24e914-0580-4621-bff3-bf545bb2bb5e",


    "Madhya Pradesh Gross State Domestic Product at Factor Cost by Industry of Origin at Current Prices - From 2004-05 to 2013-14 (as on 01.03.2014)": "/resource/f3a8d58d-7647-4bd7-ac82-2ebf2b5b3b82",
    "Puducherry Gross State Domestic Product at Factor Cost by Industry of Origin at Current Prices - From 2004-05 to 2014-15 (as on 27.02.2015)": "/resource/54be259d-7b2b-47af-a7f2-cbb98eb64941",
    "Chhattisgarh Gross State Domestic Product at Factor Cost by Industry of Origin at Current Prices - From 2004-05 to 2014-15 (as on 27.02.2015)": "/resource/99656e52-96d7-4354-bdf6-4aead8516559",
    "Odisha Gross State Domestic Product at Factor Cost by Industry of Origin at Current Prices - From 2004-05 to 2014-15 (as on 27.02.2015)": "/resource/c56a815f-b2b2-42e9-857e-6f092ba3405f",
    "Uttarakhand Gross State Domestic Product at Factor Cost by Industry of Origin at Current Prices - From 2004-05 to 2013-14 (as on 01.08.2014)": "/resource/160b3df4-ab67-47f4-a41b-100acd5cd015",
    "Jammu & Kashmir Gross State Domestic Product at Factor Cost by Industry of Origin at Current Prices - From 2004-05 to 2013-14 (as on 01.08.2014)": "/resource/0aceea80-b756-442e-9c25-44a72e7106c6",
    "Puducherry Gross State Domestic Product at Factor Cost by Industry of Origin at Current Prices - From 2004-05 to 2013-14 (as on 01.03.2014)": "/resource/0d20bd6d-09e1-4985-b0f5-cad8d3910f3b",
    "Jharkhand Gross State Domestic Product at Factor Cost by Industry of Origin at Current Prices - From 2004-05 to 2013-14 (as on 01.03.2014)": "/resource/0d14f4c2-bd05-4dd6-8f36-019abae64e20",
    "Karnataka Gross State Domestic Product at Factor Cost by Industry of Origin at Current Prices - From 2004-05 to 2013-14 (as on 01.03.2014)": "/resource/24c9212e-1206-41d9-a9cb-b97c0e2f347e",
    "Meghalaya Gross State Domestic Product at Factor Cost by Industry of Origin at Current Prices - From 2004-05 to 2013-14 (as on 01.03.2014)": "/resource/cc062471-8ca6-4770-b24c-c2677f713f16",
    "Gross State Domestic Product of Meghalaya at factor cost at current prices for the base year 2004-05": "/resource/7cc131aa-8280-438e-8096-becd8832656c",
    "Gross State Domestic Product of Chandigarh at factor cost at current prices for the base year 2004-05": "/resource/1f8bf1e9-8e8d-4abc-842b-e12ae04a58a0",
    "Gross State Domestic Product of Goa at factor cost at current prices for the base year 2004-05": "/resource/f50bc195-f652-4257-b45f-928a3be9eba1",
    "Gross State Domestic Product at Factor Cost by Industry of Origin at Current Prices for Gujarat from 2004-05 to 2013-14 (as on 31.07.2015)": "/resource/7803d463-b04f-4fe9-87e3-4b1737f21c4c",
    "Karnataka Gross State Domestic Product at Factor Cost by Industry of Origin at Current Prices - From 2004-05 to 2013-14 (as on 27.02.2015)": "/resource/2b7d6fa1-9bf9-4f8a-a611-a37e6f0df984",
    "Karnataka Gross State Domestic Product at Factor Cost by Industry of Origin at Current Prices - From 2004-05 to 2013-14 (as on 01.08.2014)": "/resource/ef16996d-f298-4f48-8eb0-00f58ea81612",
    "Tripura Gross State Domestic Product at Factor Cost by Industry of Origin at Current Prices - From 2004-05 to 2013-14 (as on 27.02.2015)": "/resource/a2ef7b6c-c1da-4bc6-8231-a2deca8439f0",
    "Nagaland Gross State Domestic Product at Factor Cost by Industry of Origin at Current Prices - From 2004-05 to 2013-14 (as on 01.03.2014)": "/resource/8cd60dd8-0b9d-44cb-b972-4a8c9f2fdfe8",
    "Delhi Gross State Domestic Product at Factor Cost by Industry of Origin at Current Prices - From 2004-05 to 2013-14 (as on 27.02.2015)": "/resource/104ce858-7102-447f-95ef-d7b011d6e144",
    "Kerala Gross State Domestic Product at Factor Cost by Industry of Origin at Current Prices - From 2004-05 to 2013-14 (as on 27.02.2015)": "/resource/b12e35d4-0c6b-47a7-8614-aa35e85340ab",
    "GSVA by Economic Activity at Constant (2011-12) Prices for Tamil Nadu from 2011-12 to 2016-17 (as on 31.03.2017)": "/resource/3b410826-eb17-42b3-aa06-8f175dc1d343",
    "GSVA by Economic Activity at Constant (2011-12) Prices for Chhattisgarh from 2011-12 to 2016-17 (as on 31.03.2017)": "/resource/951d1131-7dd2-4e3f-914a-fc3ca67c17d7",
}

In [24]:
governance_and_administration_data_apis = {
    "Attendance of the Members of Lok Sabha 15 for Winter Session 3": "/resource/97b9b3dd-c9ae-4f0b-8bdc-1c1a93655b0e",
    "Attendance of the Members of Lok Sabha 15 for Budget Session 2": "/resource/4ca4bd98-5ced-4be9-bb16-85245b479e83",
    "Attendance of the Members of Lok Sabha 15 for the Special Session 1": "/resource/01915135-30b6-4e32-8e8f-36a9f683213b",
    "Department-wise receipts, disposal and pendency of Public Grievance detailed statistics from 01.01.2016 to 01.11.2019": "/resource/1d369aae-155a-4cc8-b7a8-04d4cd5ec2a6",
    "District wise allocation of fund under State Balanced Growth Fund by various indicators in Tamil Nadu in 2013-14": "/resource/656cdf9e-cd82-4778-b99b-79593dd4f456",
    "List of advocates appointed by the Central Government in Maharashtra as on October 2016": "/resource/cae975f1-5bde-4186-bc0e-4cb3e09b5995",
    "List of Services offered through Karnataka One center": "/resource/60611598-99e0-4b76-9e64-4c0187e74aba",
    "Members of Tenth Lok Sabha from 20th June 1991 to 10th May 1996": "/resource/66d0a019-b697-46dc-8d8b-96e03ef80975",
    "District wise Akshaya Centre List in Kerala as on 13th May 2019": "/resource/c9897cec-fa66-47fc-838b-2f53ab20f086",
    "Visitors to the Visitors Gallery of Lok Sabha in Each Session of 15th Lok Sabha": "/resource/cd079600-026b-46d9-9af1-6ac5944b008c",
    "Food Distribution Centers 26/06/2020 PCMC": "/resource/7c13ead5-f087-4b32-9f2f-4a587e467d58",
    "Highlights of Parliamentary Constituency (PC) during 2024" :"/resource/a27ba4e9-73c2-40d1-90b2-41d71ea7c283",
    "Income And Expenditure of Corporations During 2000-01 To 2011-12": "/resource/19e2bbd7-269c-481e-984d-f3e315290990",
    "Income And Expenditure Of Municipal Corporations upto 2012-13": "/resource/3b65bd92-f57b-4a27-8a52-dcfd9acfd98b",
    "Status Of Political Parties In Election To The Lok Sabha (House of people), 2009 - States": "/resource/23ea730c-b95e-4bf2-bcc2-58cc118726dc",
    "List of Notary Officers of Gujarat upto August 2016": "/resource/96d3bae0-135a-4927-8a07-c431fef889be",
    "State/UT-wise List of Successful Candidate during 2024": "/resource/194d454f-3ea8-4621-a915-b211c66e46a7",
    "State/UT-wise Number of Candidates Per Constituency during 2024": "/resource/0bd877c0-031d-49da-a743-d102dec6e7b7",
    "Party type-wise Summary Statement during 2024": "/resource/82be928f-4679-47f5-9512-2d06cf503212",
    "List of Assembly Constituencies in Meghalaya": "/resource/3f09b8c8-7fe1-4170-8961-e270dfe95642",
    "CMA Regional Offices and Urban Local Bodies (ULBs) Address Details": "/resource/06d1c774-4b82-42a2-ade3-4e8a90f1ec08",
    "State/UT-wise Number of Electors during 2024": "/resource/1a0a8469-9e7e-4cc7-9f20-5f16ab973cbd",
    "State-wise Voters Information during 2024": "/resource/f7f1bf09-7633-4474-96b2-62630c70f33c",
    "CPGRAMS Nodal Public Grievance Officers Details as on 30th October 2019": "/resource/895ee1b9-6773-403c-be40-c25738fb0489",
    "Monthly Department-wise public grievance receipts and disposals from January 2016 to Octorber 2019": "/resource/3d9192e2-f608-4bad-a5d3-b7dcee256992",
    "List of advocates appointed by the Central Government in Tamil Nadu from 2013 to June 2014": "/resource/492f1bd5-333e-4838-b474-f56889f4f582",
    "List of advocates appointed by the Central Government in Tamil Nadu till 2012": "/resource/10205f8f-5c8e-41ad-ae2b-dfe89189de7e",
    "List of advocates appointed by the Central Government in Madhya Pradesh as on April 2016": "/resource/efe86f06-3879-49a6-831f-f9a93562c0e7",
    "List of Notary Officers of Uttar Pradesh upto September 2016": "/resource/9c69b712-0ead-431f-835f-686a23adb757",
    "List of Notary Officers of Maharashtra upto August 2016": "/resource/a69d4f7b-f8fe-494e-b199-b50c3072accf",
    "List of Notary Officers of Andhra Pradesh upto August 2016": "/resource/7dbed76f-68c2-4dcc-b076-48e263b4fdc5",
    "List of Notary Officers of Karnataka upto August 2016": "/resource/f5a061a8-9039-478a-a1a0-2152b8c7dcb1",
    "Sakala District Wise Applications Details as on November 2021": "/resource/fcf2aaf1-4ece-4727-9199-f81e6b4d18e6",
    "All India level Number of Elected Representatives of Panchayati Raj Institutions from 2001 to 2014": "/resource/198c818e-1c18-4622-b694-64e6ac87aba4",
    "All India level Number of Panchayati Raj Institutions and Elected Representatives from 2001 to 2014": "/resource/0d9b06d2-16d4-4a2b-b425-3cacf6c3abad",
    "Vehicles and Land used for Solid Waste Management": "/resource/af65eaab-32d3-4489-a462-547572293218",
    "Election To The Lok Sabha (House of people), 2014 - States": "/resource/15e30591-f353-4191-b6a3-337770410258",
    "All India level Number of Districts/ DRDAs/ Blocks/ Villages from 2000 to 2012": "/resource/d644241f-72bd-4b7f-8d58-df49dba5e06e",
    "Status Of Political Parties In Election To The Lok Sabha (House of people) upto 2014 - India": "/resource/7385bb09-0ce3-44c0-bebf-dfcd240e85ca",
    "Status of Political Parties in Election to the Lok Sabha (House of people), 2014 - States": "/resource/609e490b-60e1-4654-875e-eb57eba834c8",
    "Progress on development of Parks in various ULBs (year-wise) under Atal Mission for Rejuvenation and Urban Transformation(AMRUT)": "/resource/e7d89202-c886-427b-91da-d6c096d37301",
}


In [25]:
industry_data_apis = {
# tourism - only 4 APIs!
    "Number of Hotels Rooms in India upto 2012": "/resource/aba86aa1-bbf5-4bb4-b60d-e20b9d9ca0aa",
    "Number of Hotels in India upto 2012" : "/resource/9a93c652-a168-412e-aea2-2ba67210af80",
    "Monthly Visitors Arrivals upto May 2014": "/resource/6bcb1165-84f0-44d0-a6d9-e1e1319112d6",
    "Tourism Revenue upto May 2014": "/resource/41b7e891-a4ca-4be7-8023-dde41ec77112",
# small and micro industries - High Value Datasets (HVDs)
    "Karnataka Udyam Registration Details July 2020": "/resource/5c491d1b-c7fc-4c2e-8439-ce58e33df258",
    "DISTRICT WISE TOTAL NUMBER OF ENTERPRISES AND EMPLOYMENT FOR THE MONTH OF DECEMBER 2022": "/resource/340106f3-3ae8-4869-a00d-26c257a242a0",
    "Industry group-wise distribution of number of EM-II filed by the MSMEs at DICs under various State/UT Commissionerates/Directorates of Industries during 2007-08 to 2014-15 (Provisional)": "/resource/792f1b9b-1c70-4c74-bd59-b2c27dbf9df9",
    "karnataka udyam registration units with employment details district wise": "/resource/53de5fd2-1273-4f60-8ce1-5d0a6ac04fcc", # high value dataset
    "Karnataka Udyam Registration Details June 2022": "/resource/4c9ba273-0b43-42cb-9e2f-53fac8eb8d48",
    "State-Wise Distribution Of Principal Characteristics (Fourth All India Census of MSME: Registered Sector)": "/resource/d1ea440b-fb8b-444e-9d00-153f31d6e3db",
    "List of MSME Registered Units under UDYAM": "/resource/8b68ae56-84cf-4728-a0a6-1be11028dea7",
    "State-Wise Distribution Of Working Enterprises By Type Of EnterpriseFourth All India Census of MSME 2006-07 : Registered Sector)": "/resource/3beaa7cf-279d-440c-a3c1-f8e5b8e3d550",
    "State-wise distribution of enterprises by status of operation (Fourth All India Census of MSME)": "/resource/6256cf88-1cf8-49e9-bc2e-6bffdef4f2d6",
    "State Wise Distribution Of Working Enterprises By Religion Of The Owner (Fourth All India Census of MSME 2006-07 : Registered Sector)": "/resource/e1013d45-3e2f-4a64-be9f-8cc10aef1bb1",
    "Industry Wise Distribution Of Working Enterprises By Type Of Fuel Used Fourth All India Census of MSME 2006-07 : Registered Sector)": "/resource/c36962eb-706d-4c89-9029-65701f6edff9",
    "State-Wise Distribution Of Working Enterprises By Main Source Of Energy (Fourth All India Census of MSME 2006-07 : Registered Sector)": "/resource/7c057144-b3b2-4737-b763-dcab864a06e3",
    "Industry-Wise Distribution Of Working Enteprprises By Type Of Enteprprise (At 2-Digit Level Of Nic-2004) (Fourth All India Census of MSME 2006-07 : Registered Sector)": "/resource/7b9a701f-fae5-4ba5-9bbf-136bc443df0b",
    "District Wise Total MSME Registered Enterprises under UDYAM Registration till last date": "/resource/f8cd85a1-f9b8-4ff1-b195-9f75c10eb338",
    "District Level Manufacturing MSME Registered Enterprises under UDYAM Registration till last date": "/resource/ae789703-638a-4942-b79e-17868103dec8",
    "District wise Services MSME Registered Enterprises under UDYAM Registration till last date": "/resource/c3dfe7e6-0cfd-4ddb-8f79-9cb3695d9866",
    "District and Social Category wise Total MSME Registered Enterprises till last date": "/resource/d8bf9e56-c6fa-4a23-a48f-de6df37258bf",
    "District and Social Category wise Total MSME Registered Service Enterprises till last date": "/resource/96733b11-35e9-43c0-b9f8-c20626c83879",
    "District Wise Total MSME Registered Manufacturing Enterprises till last date": "/resource/201b66f2-7fda-40b8-b613-ffb7789c4341",
    "District Wise Total MSME Registered Manufacturing and Service Enterprises till last date": "/resource/925bb7dd-50f0-4876-8a1d-a5e45c4a989a",
    "District Wise Total MSME Registered Enterprises till last date": "/resource/47d1657d-4183-46ed-bfc2-0b9a5694a742",
    "District Wise Total MSME Registered Service Enterprises till last date": "/resource/d76a86b1-6a2a-4ab3-9201-cb9f6bc61fa4",
    "District and Social Category wise Total MSME Registered Manufacturing Enterprises till last date": "/resource/4a291435-dcde-4d71-8dd2-74e6138c6b4f",
    "State Wise Distribution of Enterprises by Status of Operation, 2006-07": "/resource/906282c5-28ca-45ad-997b-3b75eb68d741",
    "State Wise Distribution Of Enterprises By Nature Of Operation (Fourth All India Census of MSME: Unregistered Sector)": "/resource/38af0182-4329-4145-9ae0-d2135f287eb1",
    "State-Wise Distribution Of Enterprises By Nature Of Activity (Fourth All India Census of MSME: Unregistered Sector)": "/resource/ebd6189d-8b6f-43e2-8bcc-96b40f029415",
    "State-Wise Distribution Of Enterprises By Type Of Enterprise (Fourth All India Census of MSME: Unregistered Sector)": "/resource/802bc38b-06b2-4e18-b866-b86ed0bc3f9d",
    "Industry-Wise Distribution Of Enterprises By Type Of Enterprise (Fourth All India Census of MSME: Unregistered Sector)": "/resource/b5dec9f5-42e9-4a75-845b-d947ebae47ec",
    "State Wise Distribution Of Enterprises By Type Of Organisation (Fourth All India Census of MSME: Unregistered Sector)": "/resource/fb044d2b-eaac-4222-a867-9b090f1144a1",
    "Industry-Wise Distribution Of Enterprises By Type Of Organisation (Fourth All India Census of MSME: Unregistered Sector)": "/resource/b9087879-4497-455b-89d6-4fba8587e709",
    "State-Wise Enterpreneurship Profile : Enterprises Owned by (Fourth All India Census of MSME: Unregistered Sector)": "/resource/7e8402f8-3aa6-4da1-9043-adbdae32e8b0",
    "State-Wise Distribution Of Enterprises By Religion Of The Owner (Fourth All India Census of MSME: Unregistered Sector)": "/resource/cda6ba5b-d251-479f-adde-af3ec86934bc",
    "State-Wise Distribution Of Enterprises By Main Source Of Energy (Fourth All India Census of MSME 2006-07 : Unregistered Sector)": "/resource/05dee3a8-cad5-4b35-8b8b-56b7c7a9d24f",
    "Industry-Wise Distribution Of Enterprises By Main Source Of Energy (Fourth All India Census of MSME 2006-07 : Unregistered Sector)": "/resource/1c2d0958-9f7f-4dbe-8872-b9b07993bc53",
    "State-Wise Distribution Of Employment By Type Of Enterprise (Fourth All India Census of MSME 2006-07 : Unregistered Sector)": "/resource/cee1dd31-6ce5-424e-95c6-68d37698a37c",
    "Industry-Wise Distribution Of Employment By Type Of Enterprise (Fourth All India Census of MSME 2006-07 : Unregistered Sector)": "/resource/626f9732-a367-4735-bf34-4c7751ef2f9b",
    "State-Wise Distribution Of Employment By Type Of Sex / Social Category (Fourth All India Census of MSME 2006-07: Unregistered Sector)": "/resource/4c5b41b2-ee08-4eb4-804f-b5c413bd74ea",
    "Industry Wise Distribution Of Principal Characteristics (Fourth All India Census of MSME 2006-07: Unregistered Sector)": "/resource/d094e3ae-513e-4dd4-8304-312bc7c83ea4",
    "State Wise Distribution Of Principal Characteristics (Fourth All India Census of MSME 2006-07 : Unregistered Sector)": "/resource/6b8eeb61-a32d-4d61-a8c7-5d7308a17e18",
    "Industry-Wise Distribution Of Principal Characteristics (At 2-Digit Level Of NIC-2004) (Fourth All India Census of MSME 2006-07 : Registered Sector)": "/resource/58fe3764-1a3f-4845-90fc-731833b9c465",
    "District-wise details of SIDCO Industrial Estates in Tamil Nadu (as on 31.07.2018)": "/resource/0763457c-1bb2-4fde-a9e5-12c0c1dd79aa",
    "SIDCO Industrial Estates developed with assistance from Japan International Cooperation Agency (JICA) fund (as on 31.07.2018)": "/resource/117a7ec5-8338-44f6-a71d-9ca431b17c0f",
    "Summary Results & Comparison With Third All India Census Of SSIs 2001-02: Unregistered Sector. (Fourth All India Census of MSME: Unregistered Sector)": "/resource/c0673563-096c-4847-8c14-de616d109753",
    "Summary at a glance of Fourth All India Census of MSME : Registered Sector, 2006-07": "/resource/526aab7f-000c-4a52-9cdb-1b20c84bb6f0",
    
}


In [26]:
infrastrucutre_data_apis = {
# roads
    # length stats
        "State/Ut-wise Length of National Highways in India as on 31st March, 2013" : "/resource/c2d3f4b9-a5c3-4fe1-805b-e73150657855",
        "Total and Surfaced Road Length by Categories in India as on 31st March 2014" : "/resource/528ddbe8-f195-46c2-a4fc-5fa65341b04a",
        "State/Ut-wise Length of National Highways in India as on 31st March 2014": "/resource/7b1c5912-b5c5-4e76-82af-a5eba5503507",
        # "State/Ut-wise Length of State Highways by Type of Surface in India as on
        # 31st March 2014": # no API available
        # "State/Ut-wise Surfaced Length of State Highways by Width in India as on
        # 31st March 2014" : # no API available
        # Multiple datas exist but they does not have APIs available, we can
        # download the CSVs directly if required.
        "Total Road Length and Percentage Share of Each Category of Road to total road length from 1951 to 2017": "/resource/b4937eaa-15ec-4e64-9845-5004287f9561",
        "All India and State-wise Total And Surfaced Road Length As On 31st March 2011" : "/resource/63996d4a-e5ff-489c-9fb6-9e5118f377c0",
    # roadways investments
        # "Long Term Trends of Investment in Highways from 2004-05 to 2016-17": # downloaded
        "Investment in Highways in both Public and Private from 2012-13 to 2016-17": "/resource/e4d07942-2bd1-4167-bf91-fe4f29c82313",
    # roadways revenue
        "Year-wise Revenue Realised from Road Transport (Centre) from 2009-10 to 2018-19": "Year-wise Revenue Realised from Road Transport (Centre) from 2009-10 to 2018-19",
    
    
    # """
    # Manually downloaded:
    #     1. Total road length and percentage share of each category of road to
    #        total road length during 1951 to 2011
    #     2. Total road length and percentage share of each category of road to total road length during 1951 to 2011
        
    # """`
    


    # railways
}


In [27]:
labour_and_employment_data_apis = {
    
}


In [28]:
rural_development_data_apis = {
    
}

In [29]:

def sanitize_filename(name: str, max_len: int = 200) -> str:
    """Make a safe filename / table-name-like string."""
    safe = re.sub(r'[\\/:"*?<>|]+', '_', name)
    safe = re.sub(r'\s+', '_', safe).strip('_')
    if len(safe) > max_len:
        safe = safe[:max_len]
    return safe or "unnamed"

def _spinner(stop_event: threading.Event, text: str = "Working"):
    spinner = itertools.cycle(["|", "/", "-", "\\"])
    while not stop_event.is_set():
        sys.stdout.write(f"\r{text}... {next(spinner)}")
        sys.stdout.flush()
        time.sleep(0.12)
    sys.stdout.write("\r" + " " * 80 + "\r")
    sys.stdout.flush()

def _print_progress(completed: int, total: int):
    bar_len = 30
    filled = int(bar_len * completed / total) if total else bar_len
    bar = "#" * filled + "-" * (bar_len - filled)
    sys.stdout.write(f"\rProgress: [{bar}] {completed}/{total}")
    sys.stdout.flush()
    if completed == total:
        sys.stdout.write("\n")

def _fetch_one_api(session: requests.Session, api_name: str, api_url: str,
                   params: Dict[str, Any], json_dir: str, csv_dir: str, verify_ssl: bool):
    """
    Fetch JSON and CSV for one API and write to json_dir / csv_dir.
    Returns dict with csv text + paths.
    """
    if not api_url.startswith("http"):
        api_url = f"https://api.data.gov.in/{api_url.lstrip('/')}"
    sanitized = sanitize_filename(api_name)

    # JSON
    jr = session.get(api_url, params={**params, "format": "json"}, timeout=30, verify=verify_ssl)
    jr.raise_for_status()
    json_data = jr.json()
    json_path = os.path.join(json_dir, f"{sanitized}.json")
    with open(json_path, "w", encoding="utf-8") as f:
        json.dump(json_data, f, indent=2, ensure_ascii=False)

    # CSV
    cr = session.get(api_url, params={**params, "format": "csv"}, timeout=30, verify=verify_ssl)
    cr.raise_for_status()
    csv_text = cr.text
    csv_path = os.path.join(csv_dir, f"{sanitized}.csv")
    with open(csv_path, "w", encoding="utf-8", newline="") as f:
        f.write(csv_text)

    return {"api_name": api_name, "csv": csv_text, "json_path": json_path, "csv_path": csv_path}


In [30]:
def get_data_from_api(
    data_apis: Dict[str, str],
    api_key: str,
    dataset_label: str,
    max_workers: int = 8,
    limit: int = 999,
    json_root: str = "_raw_json_data",
    csv_root: str = "_raw_csv_data",
    verify_ssl: bool = True,
) -> Dict[str, Dict[str, Any]]:
    """
    Concurrently fetch JSON+CSV for each API in `data_apis` and save files
    under <json_root>/<dataset_label>/ and <csv_root>/<dataset_label>/.

    Args:
      data_apis: dict mapping api_name -> api_path_or_url (the same form you used before).
      api_key: data.gov.in API key.
      dataset_label: short label for this group (e.g., "agriculture", "water").
      max_workers: concurrency level for downloads.
      limit: API 'limit' parameter (as string in requests).
      json_root / csv_root: root folders under which per-dataset folders will be created.
      verify_ssl: whether to verify TLS certs in requests.

    Returns:
      mapping api_name -> {"csv": raw_csv_text, "json_path": path, "csv_path": path}
    """
    # create dataset-specific directories
    json_data_dir = os.path.join(sanitize_filename(dataset_label), json_root)
    csv_data_dir = os.path.join(sanitize_filename(dataset_label), csv_root)
    os.makedirs(json_data_dir, exist_ok=True)
    os.makedirs(csv_data_dir, exist_ok=True)

    params = {"api-key": api_key, "limit": str(limit)}
    extracted: Dict[str, Dict[str, Any]] = {}

    total_tasks = len(data_apis)
    completed = 0
    lock = threading.Lock()

    stop_event = threading.Event()
    spinner_thread = threading.Thread(target=_spinner, args=(stop_event, f"Fetching {dataset_label} APIs"), daemon=True)
    spinner_thread.start()

    with requests.Session() as session:
        session.headers.update({"Accept": "application/json, text/csv;q=0.9, */*;q=0.1"})
        with ThreadPoolExecutor(max_workers=max_workers) as exe:
            future_to_name = {
                exe.submit(_fetch_one_api, session, name, url, params, json_data_dir, csv_data_dir, verify_ssl): name
                for name, url in data_apis.items()
            }
            try:
                for fut in as_completed(future_to_name):
                    api_name = future_to_name[fut]
                    try:
                        res = fut.result()
                        extracted[api_name] = {"csv": res["csv"], "json_path": res["json_path"], "csv_path": res["csv_path"]}
                        msg = f"Done: {api_name} (saved: {res['json_path']}, {res['csv_path']})"
                    except Exception as exc:
                        extracted[api_name] = {"error": str(exc)}
                        msg = f"Error for {api_name}: {exc}"
                    with lock:
                        completed += 1
                        _print_progress(completed, total_tasks)
                        sys.stdout.write(f"{msg}\n")
                        sys.stdout.flush()
            finally:
                stop_event.set()
                spinner_thread.join()

    return extracted


In [31]:
try:
    """
    data_apis: Dict[str, str],
    api_key: str,
    dataset_label: str,
    max_workers: int = 8,
    limit: int = 999,
    json_root: str = "_raw_json_data",
    csv_root: str = "_raw_csv_data",
    verify_ssl: bool = True,
    """
    # get_data_from_api(crime_data_apis, api_key, dataset_label="crime_data")
    # get_data_from_api(agriculture_data_apis, api_key, dataset_label="agriculture_data")
    # get_data_from_api(water_data_apis, api_key, dataset_label="water_data")
    # get_data_from_api(education_data_apis, api_key, dataset_label="education_data")
    # get_data_from_api(economy_data_apis, api_key, dataset_label="economy_data")
    # get_data_from_api(governance_and_administration_data_apis, api_key, dataset_label="governance_and_administration_data")
    # get_data_from_api(industry_data_apis, api_key, dataset_label="industry_data")
    
    get_data_from_api(infrastrucutre_data_apis, api_key, dataset_label="infrastructure_data")
    # get_data_from_api(labour_and_employment_data_apis, api_key, dataset_label="labour_and_employment_data")
    # get_data_from_api(rural_development_data_apis, api_key, dataset_label="rural_development_data")
except Exception as e:
    print(f"Some error occured in getting the data from data.gov.in")
    print(f"The error is {e}")
    

Progress: [####--------------------------] 1/7Error for Year-wise Revenue Realised from Road Transport (Centre) from 2009-10 to 2018-19: 404 Client Error: Not Found for url: https://api.data.gov.in/Year-wise%20Revenue%20Realised%20from%20Road%20Transport%20(Centre)%20from%202009-10%20to%202018-19?api-key=579b464db66ec23bdd000001948d55cf357f47cc74b80568eae5c51a&limit=999&format=json
Progress: [########----------------------] 2/7Done: All India and State-wise Total And Surfaced Road Length As On 31st March 2011 (saved: infrastructure_data/_raw_json_data/All_India_and_State-wise_Total_And_Surfaced_Road_Length_As_On_31st_March_2011.json, infrastructure_data/_raw_csv_data/All_India_and_State-wise_Total_And_Surfaced_Road_Length_As_On_31st_March_2011.csv)
Progress: [############------------------] 3/7Done: State/Ut-wise Length of National Highways in India as on 31st March, 2013 (saved: infrastructure_data/_raw_json_data/State_Ut-wise_Length_of_National_Highways_in_India_as_on_31st_March,_201